In [40]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [41]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=2, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [42]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [43]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)

# volume stuff
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

# bc prep
bc_tx = CoefficientFunction(x)
bc_0x = 1/surf_0*Integrate(bc_tx,mesh)

# bc constraints
beta0 = 1e-3
beta = Parameter(beta0)
bc_x = Parameter(1)
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))

dJOmega += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)

In [44]:
dJOmega.Assemble()
print("curvol: ", surf_0)
print("Initial bc: ", bc_x.Get())
print(dJOmega.vec[0:5])

curvol:  12.566430839284152
Initial bc:  -5.080028269950915e-17
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [49]:
mesh.UnsetDeformation()
test = CoefficientFunction((0.8,1.2))
gfX.Set(test)
mesh.SetDeformation(gfX)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [50]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
dJOmega.Assemble()
print("curvol: ", Integrate(1,mesh))
print(bc_x.Get())
print(abs(bc_0x-bc_x.Get()))
print(dJOmega.vec)

curvol:  12.56643083928415
0.7999999999999996
0.7999999999999996
  1.89735e-19
 -0.000392056

  0.000392056
 1.35525e-19

  -4.40457e-20
 0.000392056

  -0.000392056
 -8.80914e-20

  0.000123018
 -0.0003812

  0.000235182
 -0.000324147

  0.000324147
 -0.000235182

  0.0003812
 -0.000123018

  0.0003812
 0.000123018

  0.000324147
 0.000235182

  0.000235182
 0.000324147

  0.000123018
 0.0003812

  -0.000123018
 0.0003812

  -0.000235182
 0.000324147

  -0.000324147
 0.000235182

  -0.0003812
 0.000123018

  -0.0003812
 -0.000123018

  -0.000324147
 -0.000235182

  -0.000235182
 -0.000324147

  -0.000123018
 -0.0003812

  6.77626e-21
 -1.0842e-19

  -8.13152e-20
      0

  -1.0842e-19
 -2.71051e-20

  6.94567e-20
 1.0842e-19

       0
 -1.35525e-19

  5.42101e-20
 -2.71051e-20

  -1.35525e-20
 1.0842e-19

  -5.42101e-20
 1.35525e-20

  5.42101e-20
 -8.13152e-20

  2.71051e-20
 -2.71051e-20

  1.0842e-19
 -1.81053e-20

       0
 5.42101e-20

  2.71051e-20
 1.0842e-19

  -5.42101e-20
 -

## try to do this as shape optimization

$\Large \frac{\partial }{\partial x} \beta(bc(x)-bc_0)^2 =$ <br>
$\Large 2(\frac{1}{bc(x)}-bc_0)\cdot \frac{\partial }{\partial x}(\frac{1}{bc(x)})$

In [88]:
# don't use 2 deformations, just set bc_0x to something else, let's see if we get there:
mesh.UnsetDeformation()
bc_0x = 3

gfset = GridFunction(VEC)

dJOmega2 = LinearForm(VEC)
#dJOmega2 += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)
dJOmega2 += ((2*beta*((1/bc_x-bc_0x))*(1/bc_x)*dx)).DiffShape(X)

gfset = GridFunction(VEC)

b = BilinearForm(VEC)
b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega2.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [89]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
bc_x.Get()

-5.080028269950915e-17

In [90]:
import time
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 100

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh)) # needed for multiple iterations (already done once before)
    print(bc_x.Get())
    dJOmega2.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.1 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    
    time.sleep(0.1)

-5.080028269950915e-17
1.4006351566346755e-05
0.0023193158109753153
0.002286869875901136
0.002255452077417368
0.0022235148028596405
0.00219110385621081
0.002158263313539249
0.0021250355719572294
0.002091461398338092
0.0020575799777029025
0.0020234289611910997
0.0019890445135384684
0.0019544613599879143
0.0019197128325665154
0.0018848309156665125
0.0018498462908744713
0.0018147883809975004
0.0017796853932402917
0.0017445643614938547
0.0017094511876992512
0.0016743706822562945
0.0016393466034519313
0.0016044016958856437
0.0015695577278765395
0.0015348355278375254
0.0015002550196091613
0.001465835256746472
0.0014315944557583588
0.0013975500282996445
0.0013637186123214584
0.0013301161021864343
0.0012967576777593124
0.001263657832484508
0.0012308304004658194


KeyboardInterrupt: 

In [ ]:
bc_x.Get()